# Autoencoder Formulation for Pareto Front Analysis

## Data Preparation
**Input Data Structure**:
- Let $X = \{\mathbf{x}_i\}_{i=1}^N \subset \mathbb{R}^2$ be the set of Pareto-optimal solutions
- Each solution $\mathbf{x}_i = (f_1^{(i)}, f_2^{(i)})$ represents a trade-off between:
  - $f_1$: Travel time (minutes)
  - $f_2$: Energy consumption (kWh)


**Normalization**: Scale objectives to $[0,1]$ range for training stability:
$$
\hat{f}_1 = \frac{f_1 - f_{1}^{min}}{f_{1}^{max} - f_{1}^{min}}, \quad
\hat{f}_2 = \frac{f_2 - f_{2}^{min}}{f_{2}^{max} - f_{2}^{min}}
$$

**Standardization**:
$$
\hat{f}_k^{(i)} = \frac{f_k^{(i)} - \mu_k}{\sigma_k} \quad \text{for } k=1,2
$$
where $\mu_k$, $\sigma_k$ are the mean and standard deviation of each objective.

## Neural Architecture Specification

### Encoder Network (Compression)
$$
\mathbf{z} = g_\phi(\mathbf{x}) = \text{LeakyReLU}(\mathbf{W}_2 \cdot \text{ELU}(\mathbf{W}_1\mathbf{x} + \mathbf{b}_1) + \mathbf{b}_2)
$$

### Decoder Network (Reconstruction)
$$
\hat{\mathbf{x}} = f_\theta(\mathbf{z}) = \text{Sigmoid}(\mathbf{W}_4 \cdot \text{ELU}(\mathbf{W}_3\mathbf{z} + \mathbf{b}_3) + \mathbf{b}_4)
$$

**Dimensionality**:
- Input/Output: $\mathbb{R}^2$ (normalized objectives)
- Latent space: $\mathbb{R}^1$ (bottleneck)
- Hidden layers: 32 neurons with ELU activation


## Architecture

### Encoder
Maps 2D Pareto solutions to a 1D latent space:
$$
\mathbf{z} = \text{Encoder}(\mathbf{x}) = \sigma(\mathbf{W}_2 \cdot \text{ReLU}(\mathbf{W}_1\mathbf{x} + \mathbf{b}_1) + \mathbf{b}_2)
$$

Where:
- $\mathbf{W}_1 \in \mathbb{R}^{h \times 2}$, $\mathbf{W}_2 \in \mathbb{R}^{1 \times h}$ are weight matrices
- $\mathbf{b}_1 \in \mathbb{R}^h$, $\mathbf{b}_2 \in \mathbb{R}^1$ are bias terms
- $h$ is hidden layer size
- $\sigma$ is sigmoid activation

### Decoder
Reconstructs solutions from latent space:
$$
\hat{\mathbf{x}} = \text{Decoder}(\mathbf{z}) = \sigma(\mathbf{W}_4 \cdot \text{ReLU}(\mathbf{W}_3\mathbf{z} + \mathbf{b}_3) + \mathbf{b}_4)
$$

With:
- $\mathbf{W}_3 \in \mathbb{R}^{h \times 1}$, $\mathbf{W}_4 \in \mathbb{R}^{2 \times h}$
- $\mathbf{b}_3 \in \mathbb{R}^h$, $\mathbf{b}_4 \in \mathbb{R}^2$

### Loss Function
Mean Squared Error (MSE) reconstruction loss:
$$
\mathcal{L}_{recon} = \frac{1}{N}\sum_{i=1}^N \|\mathbf{x}_i - \hat{\mathbf{x}}_i\|^2_2
$$

## Interpolation in Latent Space

### Linear Interpolation
For solutions $\mathbf{x}_A$, $\mathbf{x}_B$:
1. Encode two solutions:
   $$
   \mathbf{z}_A = \text{Encoder}(\mathbf{x}_A), \quad \mathbf{z}_B = \text{Encoder}(\mathbf{x}_B)
   $$

2. Linear interpolation:
   $$
   \mathbf{z}_{new} = \alpha\mathbf{z}_A + (1-\alpha)\mathbf{z}_B, \quad \alpha \in [0,1]
   $$

3. Decode to generate new solution:
   $$
   \mathbf{x}_{new} = \text{Decoder}(\mathbf{z}_{new})
   $$


### Geodesic Interpolation
For solutions $\mathbf{x}_A$, $\mathbf{x}_B$:
1. Encode: $\mathbf{z}_A = g_\phi(\mathbf{x}_A)$, $\mathbf{z}_B = g_\phi(\mathbf{x}_B)$
2. Spherical interpolation:
   $$
   \mathbf{z}_{\text{new}} = \frac{\sin[(1-\alpha)\Omega]}{\sin\Omega}\mathbf{z}_A + \frac{\sin[\alpha\Omega]}{\sin\Omega}\mathbf{z}_B
   $$
   where $\Omega = \arccos(\mathbf{z}_A^\top \mathbf{z}_B)$

3. Decode: $\mathbf{x}_{\text{new}} = f_\theta(\mathbf{z}_{\text{new}})$


## Solution Validation Protocol

### Dominance Verification
$$
\mathbf{x}_{\text{new}} \text{ is non-dominated iff } \nexists \mathbf{x}_i \in X :
\begin{cases}
f_1^{(i)} \leq f_1^{\text{(new)}} \\
f_2^{(i)} \leq f_2^{\text{(new)}} \\
\|\mathbf{x}_i - \mathbf{x}_{\text{new}}\|_2 > \delta
\end{cases}
$$

### Feasibility Check
$$
\mathbf{x}_{\text{new}} \in \mathcal{F} \iff
\begin{cases}
g_1(\mathbf{x}_{\text{new}}) \leq 0 \\
g_2(\mathbf{x}_{\text{new}}) \leq 0 \\
\vdots \\
g_k(\mathbf{x}_{\text{new}}) \leq 0
\end{cases}
$$

Ensure:
$$
f_1^{new} \geq 0, \quad f_2^{new} \geq 0
$$
And any problem-specific constraints (e.g., $g(\mathbf{x}_{new}) \leq 0$)


## Implementation Notes

1. **Normalization**: Essential for stable training
2. **Bottleneck Size**: 1D latent space enables linear interpolation
3. **Activation**: Sigmoid ensures outputs stay in normalized $[0,1]$ range
4. **Regularization**: Consider adding KL divergence for variational AE

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalize to [0, 1]
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(pareto_front)

# Split data (80% train, 20% validation)
X_train, X_val = train_test_split(X_normalized, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Define autoencoder architecture
input_dim = 2
latent_dim = 1  # 1D latent space for simplicity

# Encoder
inputs = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(inputs)
encoded = Dense(latent_dim, activation='linear')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Compile
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train
history = autoencoder.fit(
    X_train, X_train,
    epochs=500,
    batch_size=16,
    validation_data=(X_val, X_val),
    verbose=1
)

In [ ]:
# Encode two Pareto solutions
z_A = encoder.predict(X_train[0:1])  # Solution A
z_B = encoder.predict(X_train[1:2])  # Solution B

# Linear interpolation
alpha = 0.5
z_new = alpha * z_A + (1 - alpha) * z_B

# Decode to generate new solution
J_new = decoder.predict(z_new)

# Denormalize
J_new_original = scaler.inverse_transform(J_new)
print(f"Interpolated Solution: Time = {J_new_original[0, 0]:.2f} s, Energy = {J_new_original[0, 1]:.2f} kWh")